In [ ]:
import pandas as pd

In [ ]:
# from sentence_transformers import SentenceTransformer, util
# model = SentenceTransformer('all-MiniLM-L6-v2')

# #Sentences are encoded by calling model.encode()
# emb1 = model.encode("Test")
# emb2 = model.encode("Test")

# cos_sim = util.cos_sim(emb1, emb2)
# print("Cosine-Similarity:", cos_sim)
# print(cos_sim.shape)

In [ ]:
df = pd.read_csv("C:\\Users\\User\\Desktop\\nlp\\Entwicklungsdaten.tsv", sep="\t")
df1 = pd.read_csv("C:\\Users\\User\\Desktop\\nlp\\Trainingsdaten-GermEvalDatei2018.tsv", sep="\t", names=["c_text","hate","type","score"])
df2 = pd.read_csv("C:\\Users\\User\\Desktop\\nlp\\Trainingsdaten-GermEvalDatei2019.tsv", sep="\t", names=["c_text","hate","type","score"])

In [ ]:
joined_frames = [df, df1, df2] 
result = pd.concat(joined_frames)

In [ ]:
text= result.iloc[0:, 1 ] #result
print(text)

In [ ]:
tempArray = []
for row in range(len(df1)):
    #print(df1.iloc[row, 2])
    selected_row =df1.iloc[row, 2]
    if selected_row == "INSULT" or selected_row == "ABUSE":
        tempArray.append(1)
    else:
        tempArray.append(0)
#print(tempArray)

In [ ]:
df1['label'] = pd.Series(tempArray)

In [ ]:
df1

In [ ]:
tempArray = []
for row in range(len(df2)):
    #print(df1.iloc[row, 2])
    selected_row =df2.iloc[row, 2]
    if selected_row == "INSULT" or selected_row == "ABUSE":
        tempArray.append(1)
    else:
        tempArray.append(0)
#print(tempArray)

df2['label'] = pd.Series(tempArray)

In [ ]:
df2

In [ ]:
hs =  (df.iloc[0: , 8 ].to_list() + df1.iloc[0: , 4].to_list() + df2.iloc[0: , 4].to_list() ) # 

In [ ]:
print(hs)

In [ ]:
import spacy
nlp = spacy.load("de_core_news_sm")

In [ ]:
array =[]
for t in text[:]:
    doc = nlp(t)
    #print([token.text for token in doc])
    
    #normalize dataset
    
    #with stop words
    #temp = [token.lemma_.lower() for token in doc if token.is_alpha]
    
    #without stop words and only alpha
    #temp = [token.lemma_.lower() for token in doc if token.is_alpha and not token.is_stop]
    
    #without stop words and with smileys /no alpha chars
    temp = [token.lemma_.lower() for token in doc if not token.is_stop ] #and not token.text == '@'
    
    #with smileys and stopwords
    #temp = [token.lemma_.lower() for token in doc]
    
    #print(temp)

    array.append((" ".join(temp)))

print (array)

In [ ]:
df = pd.DataFrame ({'Text': array , 'Hatespeech': hs })
print (df)




In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df.Text, 
    df.Hatespeech, 
    test_size=0.2, # 20% samples will go to test dataset
    random_state=2021,#-> same order of dataset random_state=2022, 2021
    stratify=df.Hatespeech
)

In [ ]:
print("Shape of X_train: ", X_train.shape)
print("Shape of X_train: ", y_train.shape)
print("Shape of X_test: ", X_test.shape)
print("Shape of X_train: ", y_test.shape)


In [ ]:
print((X_train.tolist()))

In [ ]:
y_train.value_counts()

In [ ]:
y_test.value_counts()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.pipeline import Pipeline 
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier 
from sklearn.datasets import make_classification 
import xgboost as xgb




#tfidf
#vecktorisieren

# vectorizer = TfidfVectorizer() 
# Xtrain = vectorizer.fit_transform(X_train) 
# Xtest = vectorizer.transform(X_test) 
# print(Xtrain.shape)

#vecktorisieren 



from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader


#model = SentenceTransformer('all-MiniLM-L12-v2')
#model = SentenceTransformer('all-MiniLM-L6-v2')
#model = SentenceTransformer('all-mpnet-base-v2')
#model = SentenceTransformer('multi-qa-mpnet-base-dot-v1')
#model = SentenceTransformer('all-distilroberta-v1') multi-qa-distilbert-cos-v1
#model = SentenceTransformer('multi-qa-distilbert-cos-v1') #multi-qa-MiniLM-L6-cos-v1
#model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1') #paraphrase-multilingual-mpnet-base-v2
model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2') # best #paraphrase-albert-small-v2
#model = SentenceTransformer('paraphrase-albert-small-v2') #paraphrase-multilingual-MiniLM-L12-v2
#model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2') #paraphrase-MiniLM-L3-v2
#model = SentenceTransformer('paraphrase-MiniLM-L3-v2') #distiluse-base-multilingual-cased-v1
#odel = SentenceTransformer('distiluse-base-multilingual-cased-v1') #distiluse-base-multilingual-cased-v2
#odel = SentenceTransformer('distiluse-base-multilingual-cased-v2')

#print(X_train)
x = X_train.tolist()
Xtrain = model.encode(x) 
xt = X_test.tolist()
Xtest = model.encode(xt)



#create classifier and train model 
clf = xgb.XGBClassifier(objective="binary:logistic", random_state=2021, #booster='gblinear', 2021
                        
#                         n_estimators = 1000,
#                         gamma=0.61, #2.7
#                         min_child_weight=0.1,                         
#                         learning_rate =0.3, #0,55 with bytree 1.... 0.3/0.38 best
#                         reg_alpha=0,
#                         max_depth=12,
 
                        
#                         colsample_bytree=1, colsample_bylevel=0.5, colsample_bynode=1
                       )

clf.fit(Xtrain, y_train)

#3. get the predictions for X_test and store it in y_pred 
y_pred = clf.predict(Xtest)

#4. print the classfication report 
print(classification_report(y_test, y_pred))

#find accuracy scores
accuracy = accuracy_score(y_test, y_pred) 
print("The accuracy of prediction is: ", accuracy)

In [ ]:
print(Xtrain.shape)
print(Xtest.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
# result = classification_report(y_test, y_pred)
# dff = pd.DataFrame(result)
# dff.to_csv("xgb_score.csv", index=False)

In [ ]:
df5 = (classification_report(y_test, y_pred))
#df5 = (pd.DataFrame

# df = df.sort_values("rank_test_r2")
# df.to_csv("xgb_results.txt", index=False)


In [ ]:
print(df5)

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
import xgboost as xgb

from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer

#1. create a pipeline object
clf = Pipeline([
     ('vectorizer_tfidf',TfidfVectorizer()),    
     #('KNN', KNeighborsClassifier())
     #('SVM', SVC())   
     #('rfc', RandomForestClassifier()) 
     ('xgb', xgb.XGBClassifier(objective="binary:logistic", random_state=2021))  #42 tree_method='gpu_hist'
])

#2. fit with X_train and y_train
#clf.fit(X_train, y_train)

#rf
#  'rf__max_depth': [1000],
#  'rf__min_samples_split': [100],
#  'rf__max_leaf_nodes': [None]

# Setting up gxboost params
#'xgb__min_samples_split': [100],
    #'xgb__max_leaf_nodes': [None],
#'vectorizer_tfidf__stop_words': ['english'],
 #'vectorizer_tfidf__max_features':[2000],
 #'vectorizer_tfidf__ngram_range': [(1, 2)], 

#     "n_estimators" : [100, 200, 500],
#     "max_depth" : [3, 6, 9],
#     "gamma" : [0.01, 0.1],
#     "learning_rate" : [0.001, 0.01, 0.1, 1]
#   'xgb__colsample_bytree' : [0.3, 0.5, 0.8],
#   'vectorizer_tfidf__max_features':[100, 2000],

# l1 und l2 lambda
#max def 
#ngrams tfid 1 und 2
#  scale_pos_weight

params = {
    #'vectorizer_tfidf__ngram_range': [(1, 1)], #, (1, 2), (2, 2)
    
#      'xgb__gamma': [0.3], # 0.5, 1.4, 1.7    #, 1.4 , 1.7 also good - maybe test again ,i/10.0 for i in range(0,5) , # 1.9, 1.8,  2, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7
#      'xgb__max_depth': [12,1,6,13], #3, 4 , 6 # used 1,5,6,10,12, 14
#      'xgb__min_child_weight': [0.1], # 0.1 / 2
#      'xgb__max_delta_step': [0] ,
#      'xgb__reg_alpha': [0.5], #0, 0.1, 0.2, 0.3, 0.4, wanna test #1e-5, 1e-2, 0.1, 1, 100 | 1.9 - 2.1 - 2.0 (1, 1.2, 1.3, 1.5, 1.7, 1.9, 2.0, 2.1)
#      'xgb__reg_lambda':[0.1],
#      'xgb__colsample_bytree':[0.1],  #0.5,0.6,0.7, 0.8, 0.9, 1 i/10.0 for i in range(1,10)
#     'xgb__subsample': [0.1],
    
    
    'xgb__n_estimators': [1000], 
     'xgb__gamma': [0.61],
    'xgb__min_child_weight': [0.1],
    'xgb__reg_alpha': [0],
     'xgb__max_depth': [1,6,11,12,13,14,15,16,17],
     'xgb__colsample_bytree':[1],
     'xgb__colsample_bylevel':[0.5], 
    'xgb__colsample_bynode':[1],
    
   
    
    
    #'tfidf__use_idf': (True, False),
  #  'vectorizer_tfidf__max_n': (1, 2),
   # 'vectorizer_tfidf__max_df': (0.5, 0.75, 1.0),
    #'vectorizer_tfidf__max_features': (None, 5000, 10000, 50000),
    
#                          n_estimators = 1000,
#                         gamma=0.61, #2.7
#                         min_child_weight=0.1,                         
#                         learning_rate =0.3, #0,55 with bytree 1.... 0.3/0.38 best
#                         reg_alpha=0,
#                         max_depth=16,
#                         #process_type='update',
#                         #num_parallel_tree=3
                        
#                         colsample_bytree=1, colsample_bylevel=0.5, colsample_bynode=1
    
    
      
    #'xgb__max_cat_to_onehot': [4,8,12] #,2,4   
   
    #'xgb__learning_rate' : [0.081,0.082,0.083,0.084,0.085,0.086,0.087,0.088,0.089,0.09, 0.092], #0.05, 0.1 , 0.15, 0.2, 0.25, 0.3], |0.01, 0.03, 0.06, 0.08, 0.1, 0.15, 0.2, 0.25, 0.3
    #',    
    
    #'xgb__eta': [0.1, 0.2,0.3],
    #'xgb__scale_pos_weight':[0.1,0.2,0.3,0.4,0.5,0.6,0.7] -> lose 10%
   #  'xgb__sampling_method':[gradient_based], ->no d
    
    #'xgb__subsample': [0.9], #nodif i/10.0 for i in range(6,10)
    
    
   # 'xgb__n_estimators': [1000], #, 1500,2000
  
   
   
  # 'xgb__colsample_bylevel':[0.5], 
   # 'xgb__colsample_bynode':[1]
  
    #colsample_bytree=1, colsample_bylevel=0.5, colsample_bynode=1,
}



In [ ]:
clf.get_params().keys()

In [ ]:
# Setting up GridSearch
# Setting up GridSearch for Randomforest
gs = GridSearchCV(clf, params, 
                  cv = 5, 
                  verbose = 2,
                  n_jobs = 1, 
                  scoring = 'f1',
               #  scoring = ["r2", "neg_root_mean_squared_error"], #sklearn.metrics.SCORERS.keys() 
                                                 #scoring = ['accuracy', 'precision'],
                #  refit = "r2"
                 
)

In [ ]:
# Fitting Randomforest CV GS
gs.fit(X_train, y_train)

In [ ]:
#clf.get_params().keys()

In [ ]:
print ('Best score: %0.3f' % gs.best_score_)
print ('Best parameters set:')
best_parameters = gs.best_estimator_.get_params()
#      for param_name in sorted(parameters.keys():
#          print ('\t%s: %r' % (param_name, best_parameters[param_name]))

#     predictions = gs.predict(X_test)
#     print classification_report(y_test, predictions)

In [ ]:
print(gs.best_estimator_) # to get the complete details of the best model

In [ ]:
print(gs.best_score_)

In [ ]:
print(gs.best_params_)

In [ ]:
print(gs.best_estimator_.steps)
best_clf = gs.best_estimator_
y_pred = best_clf.predict(X_test)

#3. get the predictions for X_test and store it in y_pred
#y_pred = clf.predict(X_test)



#4. print the classfication report
accuracy = accuracy_score(y_test, y_pred) 
print(classification_report(y_test, y_pred))
print("The accuracy of prediction is: ", accuracy)

#df = pd.DataFrame(classification_report(y_test, y_pred))
#df.to_csv("xgb_score.txt", index=False)

In [ ]:
 # simple filename parser and output results
import numpy as np
  #  p = np.vectorize(lambda x: x.split('.')[0].split('/')[2])
   # util.write_predictions(y_pred, p(test.filenames).tolist(), 'tfidfvec_gs.csv')

 df = pd.DataFrame(gs.cv_results_)
 df = df.sort_values("rank_test_score")
 df.to_csv("xgb_results.txt", index=False)

# df =(classification_report(y_test, y_pred))
# df.to_csv("xgb_results.txt", index=False)

# cv_results = gs.cv_results_

# report = classification_report(y_test, y_pred, output_dict=True)
# print (report)

In [ ]:
#X_test[:100]
c =0
for i in X_test[:200]:
    print(c,"->", i)
    c+=1

In [ ]:
#y_test[0:50]
c =0
for i in y_test[0:]:
    print(c,"->", i)
    c+=1

h=0
nh=0
for i in y_test:
    if i == 1:
        #print(i)
        h+=1
    if i == 0:
        #print(i)
        nh+=1
print("count of HS: ", h) 
print("count of NS: ", nh)
        

In [ ]:
#y_pred[0:50]
c =0
for i in y_pred[0:]:
    print(c,"->", i)
    c+=1

h=0
nh=0
for i in y_pred:
    if i == 1:
        #print(i)
        h+=1
    if i == 0:
        #print(i)
        nh+=1
print("count of HS: ", h) 
print("count of NS: ", nh)    